In [41]:
import requests
import os
import json
import getpass
import tarfile
from urllib.request import urlretrieve

In [2]:
host = 'https://espa.cr.usgs.gov/api/v1/'
username = 'ozanbayiz'
password = getpass.getpass()

 ········


In [46]:
def espa_api(endpoint, verb='get', body=None, uauth=None):
    """ Suggested simple way to interact with the ESPA JSON REST API """
    auth_tup = uauth if uauth else (username, password)
    response = getattr(requests, verb)(host + endpoint, auth=auth_tup, json=body)
    print('{} {}'.format(response.status_code, response.reason))
    data = response.json()
    if isinstance(data, dict):
        messages = data.pop("messages", None)  
        if messages:
            print((json.dumps(messages, indent=4)))
    try:
        response.raise_for_status()
    except Exception as e:
        print(e)
        return None
    else:
        return data
        
def create_order(day, month, year):
    #specify date as date range strings
    start_date = "20240602"
    end_date = "20240603"
    #create list of productIDs with date range strings
    ls = ["LC09_L2SP_043031_{}_{}_02_T1".format(start_date, end_date),
          "LC09_L2SP_043032_{}_{}_02_T1".format(start_date, end_date),
          "LC09_L2SP_043033_{}_{}_02_T1".format(start_date, end_date),
          "LC09_L2SP_043034_{}_{}_02_T1".format(start_date, end_date),
          "LC09_L2SP_043035_{}_{}_02_T1".format(start_date, end_date),
          "LC09_L2SP_043036_{}_{}_02_T1".format(start_date, end_date)]
    #desired prodcuct: Normalized Burn Ratio (NBR)
    prod = ["sr_nbr"]
    order = espa_api('available-products', body=dict(inputs=ls))
    #not sure what this does
    print((json.dumps(order, indent=4)))
    projection = {'aea': {'standard_parallel_1': 29.5,
                  'standard_parallel_2': 45.5,
                  'central_meridian': -96.0,
                  'latitude_of_origin': 23.0,
                  'false_easting': 0,
                  'false_northing': 0,
                  'datum': 'nad83'}}
    #populate order dict with desired products
    for sensor in order.keys():   
        if isinstance(order[sensor], dict) and order[sensor].get('inputs'):
            if set(ls) & set(order[sensor]['inputs']):
                order[sensor]['products'] = prod
    #populate other fields of order dict  
    order['projection'] = projection
    order['format'] = 'gtiff'
    order['resampling_method'] = 'cc'
    #see order
    print((json.dumps(order, indent=4)))
    return order
    
def place_order(order):
    #place order
    print('POST /api/v1/order')
    resp = espa_api('order', verb='post', body=order)
    print((json.dumps(resp, indent=4)))
    return resp

def check_order_status(orderid):
    print(('GET /api/v1/order-status/{}'.format(orderid)))
    resp = espa_api('order-status/{}'.format(orderid))
    print((json.dumps(resp, indent=4)))

def get_order_urls(orderid):
#    orderid = resp['orderid']
    #check status of order
    check_order_status(orderid)
    #check for completed products
    print(('GET /api/v1/item-status/{0}'.format(orderid)))
    resp = espa_api('item-status/{0}'.format(orderid), body={'status': 'complete'})
    #print((json.dumps(resp[orderid], indent=4)))
    #get download URLS
    urls = [item.get('product_dload_url') for item in resp[orderid]]
    for item in resp[orderid]:
        print(("URL: {0}".format(item.get('product_dload_url'))))
    return urls

def get_backlogs():
    #list backlog orders for authenticated user
    print('GET /api/v1/list-orders')
    filters = {"status": ["complete", "ordered"]}
    resp = espa_api('list-orders', body=filters)
    #print((json.dumps(resp, indent=4)))
    return resp

def download_from_urls(urls, orderid):
    path = "../data/{}".format(orderid)
    os.mkdir(path)
    for url in urls:
        print("downloading from {}...".format(url))
        file_name = url.split('/')[-1]
        full_file_name = path + file_name
        urlretrieve(url, full_file_name)

In [23]:
order = create_order(1,2,3)

200 OK
{
    "olitirs9_collection_2_l2": {
        "products": [
            "l1",
            "sr_ndvi",
            "sr_evi",
            "sr_savi",
            "sr_msavi",
            "sr_ndmi",
            "sr_nbr",
            "sr_nbr2",
            "sr_ndsi",
            "et"
        ],
        "inputs": [
            "LC09_L2SP_043031_20240602_20240603_02_T1",
            "LC09_L2SP_043032_20240602_20240603_02_T1",
            "LC09_L2SP_043033_20240602_20240603_02_T1",
            "LC09_L2SP_043034_20240602_20240603_02_T1",
            "LC09_L2SP_043035_20240602_20240603_02_T1",
            "LC09_L2SP_043036_20240602_20240603_02_T1"
        ]
    }
}
{
    "olitirs9_collection_2_l2": {
        "products": [
            "sr_nbr"
        ],
        "inputs": [
            "LC09_L2SP_043031_20240602_20240603_02_T1",
            "LC09_L2SP_043032_20240602_20240603_02_T1",
            "LC09_L2SP_043033_20240602_20240603_02_T1",
            "LC09_L2SP_043034_20240602_20240603_02_T1",

In [47]:
order

{'olitirs9_collection_2_l2': {'products': ['sr_nbr'],
  'inputs': ['LC09_L2SP_043031_20240602_20240603_02_T1',
   'LC09_L2SP_043032_20240602_20240603_02_T1',
   'LC09_L2SP_043033_20240602_20240603_02_T1',
   'LC09_L2SP_043034_20240602_20240603_02_T1',
   'LC09_L2SP_043035_20240602_20240603_02_T1',
   'LC09_L2SP_043036_20240602_20240603_02_T1']},
 'projection': {'aea': {'standard_parallel_1': 29.5,
   'standard_parallel_2': 45.5,
   'central_meridian': -96.0,
   'latitude_of_origin': 23.0,
   'false_easting': 0,
   'false_northing': 0,
   'datum': 'nad83'}},
 'format': 'gtiff',
 'resampling_method': 'cc'}

In [48]:
resp = place_order(order)

POST /api/v1/order
201 CREATED
{
    "orderid": "espa-ozanbayiz@berkeley.edu-06052024-211910-614",
    "status": "ordered"
}


In [49]:
orderid = resp['orderid']
urls = get_order_urls(orderid)
urls

GET /api/v1/order-status/espa-ozanbayiz@berkeley.edu-06052024-211910-614
200 OK
{
    "orderid": "espa-ozanbayiz@berkeley.edu-06052024-211910-614",
    "status": "ordered"
}
GET /api/v1/item-status/espa-ozanbayiz@berkeley.edu-06052024-211910-614
200 OK


[]

In [45]:
download_from_urls(urls, orderid)

downloading from https://edclpdsftp.cr.usgs.gov/orders/espa-ozanbayiz@berkeley.edu-06052024-205810-115/LC090430342024060202T1-SC20240606015913.tar.gz...
downloading from https://edclpdsftp.cr.usgs.gov/orders/espa-ozanbayiz@berkeley.edu-06052024-205810-115/LC090430322024060202T1-SC20240606015913.tar.gz...


HTTPError: HTTP Error 404: Not Found